In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [26]:
import pandas as pd

# --- Step 1: Read Excel ---
file_path = r"E:\OrderProcssing\data\Detailed Breakdown - 20250422_DCT.xlsx"
df = pd.read_excel(file_path, skiprows=1)

# --- Step 2: Keep only rows where 'Sr No.' is not NaN ---
df_clean = df[df['Sr No.'].notna()].reset_index(drop=True)

df_copy = df_clean.copy()

# --- Step 3: Check result ---

df_copy.head(10)
#df_clean.to_csv(r"C:\Users\Admin\Desktop\DCT_PO_Cleaned.csv", index=False)


,Sr No.,Po #,Unnamed: 2,SKU #,Image,Ring Size,Gold Lock,Gold Karat,Net Wt,Finding Wt,...,Dia Value,CFP,RHD,Ceramic Charges,Setting Charges,FINDING,Unnamed: 24,Price of a single item USD,Price of a single item USD.1,Price of a single item USD.2
0,1.0,20250422.0,2503.0,RG047986,NaN,54.0,3400.0,14KT Y,1.65,0.0,...,25.75,5.5,NaN,NaN,0.25,0.0,0.4,150.07,1.0,150.0700
1,2.0,20250422.0,2506.0,ER014739,NaN,NaN,3400.0,14KT RW,1.50,0.5,...,46.00,8.0,4.0,2.0,12.50,8.0,0.8,316.73,1.0,316.7300
2,3.0,20250422.0,2507.0,RG047986,NaN,54.0,3400.0,14KT R,1.65,0.0,...,25.75,5.5,NaN,NaN,0.25,0.0,0.4,150.07,2.0,300.1400
3,4.0,20250422.0,2508.0,RG047996,NaN,54.0,3400.0,14KT W,1.95,0.0,...,68.00,5.5,3.0,1.0,5.00,0.0,0.4,242.36,1.0,242.3600
4,5.0,20250422.0,2514.0,PD017166,NaN,NaN,3400.0,14KT RW,1.95,1.2,...,18.40,6.5,3.0,1.0,2.75,25.0,0.4,236.71,1.0,236.7100
5,6.0,20250422.0,2515.0,PD017145,NaN,NaN,3400.0,14KT W,1.50,1.2,...,12.40,5.5,3.0,1.0,5.50,25.0,0.4,171.4461,1.0,171.4461
6,7.0,20250422.0,2546.0,ER015416,NaN,NaN,3400.0,14KT W,0.65,0.5,...,21.11,6.5,4.0,2.0,0.50,8.0,0.8,89.46,2.0,178.9200
7,8.0,20250422.0,2547.0,ER015416,NaN,NaN,3400.0,14KT Y,0.65,0.5,...,21.11,6.5,0.0,2.0,0.50,8.0,0.8,85.46,3.0,256.3800
8,9.0,20250422.0,2550.0,ER015240,NaN,NaN,3400.0,14KT Y,0.65,0.5,...,36.57,6.5,NaN,2.0,0.50,8.0,0.8,100.92,4.0,403.6800
9,10.0,20250422.0,2552.0,ER015415,NaN,NaN,3400.0,14KT W,0.90,0.5,...,66.22,6.5,3.0,2.0,0.50,8.0,0.8,151.48,9.0,1363.3200


In [7]:
import pandas as pd

# --- Step 1: Read Excel ---
file_path = r"E:\OrderProcssing\data\Detailed Breakdown - 20250422_DCT.xlsx"
df = pd.read_excel(file_path, skiprows=1)

# --- Step 2: Keep only rows where 'Sr No.' is not NaN ---
df_clean = df[df['Sr No.'].notna()].reset_index(drop=True)
df_copy = df_clean.copy()

# --- Step 3: Select relevant columns ---
selected_columns = ['Sr No.','Po #','Ring Size','Unnamed: 2','SKU #','Gold Karat','Dia Qlty','Price of a single item USD.1']
df_selected = df_copy[selected_columns].copy()

# --- Step 4: Rename columns ---
df_selected.rename(columns={
    'Sr No.': 'SrNo',
    'SKU #': 'StyleCode',
    'Ring Size': 'ItemSize',
    'Price of a single item USD.1': 'OrderQty',
    'Gold Karat': 'Metal',
    'Po #': 'ItemPoNo',
    'Unnamed: 2': 'SKUNo',
}, inplace=True)

# --- Step 5: Format ItemSize ---
df_selected['ItemSize'] = df_selected['ItemSize'].apply(lambda x: f'EU{int(x)}' if pd.notna(x) else '')

# --- Step 6: Reorder columns: StyleCode → ItemSize → OrderQty ---
cols = df_selected.columns.tolist()
cols.insert(1, cols.pop(cols.index('StyleCode')))
cols.insert(2, cols.pop(cols.index('ItemSize')))
cols.insert(3, cols.pop(cols.index('OrderQty')))
df_selected = df_selected[cols]

# --- Step 7: Add 'OrderItemPcs' after 'OrderQty' ---
df_selected.insert(df_selected.columns.get_loc('OrderQty') + 1, 'OrderItemPcs', value='')  

# --- Step 8: Move 'Metal' after 'OrderItemPcs' ---
metal_col = df_selected.pop('Metal')
df_selected.insert(df_selected.columns.get_loc('OrderItemPcs') + 1, 'Metal', metal_col)

# --- Step 9: Map Metal values ---
metal_mapping = {
    '14KT RW': 'G585PW',
    '14KT R': 'G585P',
    '14KT Y': 'G585Y',
    '14KT W': 'G585W',
    '18KT RW': 'G750PW',
    '18KT R': 'G750P',
    '18KT Y': 'G750Y',
    '18KT W': 'G750W'
}
df_selected['Metal'] = df_selected['Metal'].map(metal_mapping).fillna('')

# --- Step 10: Add 'Tone' column after 'Metal' ---
df_selected.insert(df_selected.columns.get_loc('Metal') + 1, 'Tone', df_selected['Metal'].str.extract(r'(PW|P|Y|W)$')[0].fillna(''))

# --- Step 11: Add new columns after 'ItemPoNo' ---
insert_pos = df_selected.columns.get_loc('ItemPoNo') + 1
for col_name in ['ItemRefNo','StockType', 'Priority', 'MakeType','CustomerProductionInstruction']:
    df_selected.insert(insert_pos, col_name, '')
    insert_pos += 1  # increment position so next column comes after

# --- Step 12: Ask user for Priority value ---
priority_input = input("Enter Priority value to assign for all rows: ")
df_selected['Priority'] = priority_input  # fill all rows

# --- Step 13: Add 'SpecialRemarks' after 'CustomerProductionInstruction' ---
special_remarks_pos = df_selected.columns.get_loc('CustomerProductionInstruction') + 1

def create_special_remarks(row):
    # Remove 'EU' from ItemSize
    size = row['ItemSize'].replace('EU','') if row['ItemSize'] else ''
    dia = row['Dia Qlty'] if pd.notna(row['Dia Qlty']) else ''
    return f"{row['SKUNo']} {row['Metal']} SZ-{size} DIA: {dia}"

df_selected.insert(special_remarks_pos, 'SpecialRemarks', df_selected.apply(create_special_remarks, axis=1))


dpi_pos = df_selected.columns.get_loc('SpecialRemarks') + 1

def map_design_instruction(tone):
    if pd.isna(tone):
        return ''
    tone = tone.strip().upper()
    if tone == 'W':
        return 'WHITE RODIUM'
    elif tone in ['Y', 'PW', 'PY','P']:
        return 'NO RODIUM'
    else:
        return ''

df_selected.insert(dpi_pos, 'DesignProductionInstruction', df_selected['Tone'].apply(map_design_instruction))

stamp_pos = df_selected.columns.get_loc('DesignProductionInstruction') + 1

# Extract numeric part (e.g., 585 or 750) from the "Metal" column
df_selected.insert(
    stamp_pos,
    'StampInstruction',
    df_selected['Metal'].str.extract(r'(\d{3})')[0].fillna('')
)

ordergroup_pos = df_selected.columns.get_loc('StampInstruction') + 1

# Insert both columns sequentially
for col_name in ['OrderGroup', 'Certificate']:
    df_selected.insert(ordergroup_pos, col_name, '')
    ordergroup_pos += 1  # move position for the next column

sku_pos = df_selected.columns.get_loc('SKUNo') + 1

new_columns = [
    'Basestoneminwt', 'Basestonemaxwt',
    'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate'
]

for col_name in new_columns:
    df_selected.insert(sku_pos, col_name, '')
    sku_pos += 1  # increment position for next column

df_selected.drop(columns=['Dia Qlty'], inplace=True) 

# --- Step 14: Check result ---
df_selected.head(10)
df_selected.to_csv(r"C:\Users\Admin\Desktop\DCT_PO_Cleaned.csv", index=False)


Enter Priority value to assign for all rows:  REG
